## TWAP (Time-Weighted Average Price)
##### (My investigation here is not finished - still a work in progress)

##### As their name suggests, TWAPs accumulate an average amount of an asset over a time interval.
##### Often there may be steady transacting while prices are flat, but then the market sees the persistent bid or offer and front runs it.
##### Here I attempt to see if there is a way to detect net TWAP-like behavior in the market
##### (We can never know for sure as we do not have inside access to participants using TWAPs to buy or sell)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
eth_data = pd.read_csv("../Desktop/crypto_data_python/binance_ethusdt_042523_15min.csv")
eth_data = eth_data[["time", "open", "high", "low", "close", "Volume", "Crypto Open Interest (Open)"]]

# Now eliminate the error "A value is trying to be set on a copy of a slice from a DataFrame":
eth_prices = eth_data.copy()
eth_prices.rename(columns={"Crypto Open Interest (Open)": "open_int"}, inplace=True)
# eth_prices["time"] = pd.to_datetime(eth_prices["time"], format="%Y-%m-%dT%H:%M:%S.%f")
eth_prices["time"] = pd.to_datetime(eth_prices["time"])
print(eth_prices.head())

                       time     open     high      low    close   Volume  \
0 2023-04-16 16:00:00-04:00  2138.92  2142.02  2120.00  2136.51  785.366   
1 2023-04-16 16:15:00-04:00  2135.99  2145.90  2135.73  2145.34  202.036   
2 2023-04-16 16:30:00-04:00  2145.35  2152.52  2142.46  2151.00  492.777   
3 2023-04-16 16:45:00-04:00  2151.42  2156.41  2143.81  2146.31  172.163   
4 2023-04-16 17:00:00-04:00  2146.36  2150.75  2135.14  2139.39  238.103   

    open_int  
0  11754.415  
1  11764.052  
2  11808.052  
3  11724.149  
4  11711.973  


#### 1. Look for price stability combined with a spike in volume

In [4]:
# For stable prices, we use a deviation of +/- 3% as a cutoff
eth_prices["mid"] = (eth_prices["low"] + eth_prices["high"]) / 2
# eth_prices["price_flat"] = np.where((eth_prices["mid"].shift(3) / eth_prices["mid"].shift(4) > 0.97) \
#     & (eth_prices["mid"].shift(3) / eth_prices["mid"].shift(4) < 1.03) \
#     & (eth_prices["mid"].shift(2) / eth_prices["mid"].shift(3)  > 0.97) \
#     & (eth_prices["mid"].shift(2) / eth_prices["mid"].shift(3)  < 1.03), True, False)
eth_prices["price_flat"] = np.where(
    (eth_prices["mid"].shift(2) / eth_prices["mid"].shift(3)  > 0.97) \
    & (eth_prices["mid"].shift(2) / eth_prices["mid"].shift(3)  < 1.03), True, False)


# Ues a volume increase of at least 30% from 15-min pd to 15-min pd as a volume increase
# eth_prices["volume_inc_1st_pd"] = eth_prices["Volume"].shift(3) / eth_prices["Volume"].shift(4)
eth_prices["volume_inc_2nd_pd"] = eth_prices["Volume"].shift(2) / eth_prices["Volume"].shift(3)
eth_prices.dropna(inplace=True)

eth_prices["volume_increases"] = np.where((eth_prices["volume_inc_1st_pd"] > 1.1) \
    & (eth_prices["volume_inc_2nd_pd"] > 1.1), True, False)
    # & (eth_prices["volume_inc_2pd"] > 1.1) & (eth_prices["volume_inc_3pd"] > 1.1), True, False)

print(eth_prices.head())

                        time     open     high      low    close   Volume  \
7  2023-04-16 17:45:00-04:00  2147.65  2155.99  2144.91  2151.23  218.529   
8  2023-04-16 18:00:00-04:00  2152.26  2152.86  2147.06  2147.80  241.043   
9  2023-04-16 18:15:00-04:00  2147.45  2148.62  2143.10  2144.22   70.747   
10 2023-04-16 18:30:00-04:00  2144.44  2148.13  2142.02  2147.85   65.306   
11 2023-04-16 18:45:00-04:00  2148.71  2150.00  2146.43  2149.49   47.186   

     open_int       mid  price_flat  volume_inc_1st_pd  volume_inc_2nd_pd  \
7   11754.273  2150.450        True           1.383009           0.815844   
8   11724.362  2149.960        True           0.815844           0.743641   
9   11720.270  2145.860        True           0.743641           1.512772   
10  11729.325  2145.075        True           1.512772           1.103025   
11  11737.254  2148.215        True           1.103025           0.293504   

    volume_increases  
7              False  
8              False  
9    

#### 2. Look for higher lows or lower highs in the next period

In [5]:
# Higher Lows
conditions_hl = (eth_prices["low"].shift(-1) > eth_prices["low"]) \
    & (eth_prices["low"].shift(-2) > eth_prices["low"].shift(-1)) \
    # & (eth_prices["low"].shift(2) > eth_prices["low"].shift(3))

eth_prices["higher_lows"] = np.where(conditions_hl, True, False)

# Lower Highs
conditions_lh = (eth_prices["high"].shift(-1) > eth_prices["high"]) \
    & (eth_prices["high"].shift(-2) > eth_prices["high"].shift(-1)) \
    # & (eth_prices["high"].shift(2) > eth_prices["high"].shift(3))

eth_prices["lower_highs"] = np.where(conditions_lh, True, False)

print(eth_prices.tail())

                         time     open     high      low    close   Volume  \
831 2023-04-25 07:45:00-04:00  1839.19  1843.59  1838.12  1839.11   98.867   
832 2023-04-25 08:00:00-04:00  1839.10  1840.42  1837.23  1838.16   58.928   
833 2023-04-25 08:15:00-04:00  1837.98  1843.99  1837.95  1839.00  103.814   
834 2023-04-25 08:30:00-04:00  1839.73  1839.73  1834.06  1836.92  129.473   
835 2023-04-25 08:45:00-04:00  1836.66  1837.93  1834.93  1837.59   31.501   

     open_int       mid  price_flat  volume_inc_1st_pd  volume_inc_2nd_pd  \
831  9237.620  1840.855        True           2.551227           0.274250   
832  9236.827  1838.825        True           0.274250           3.613712   
833  9240.074  1840.970        True           3.613712           0.400622   
834  9226.574  1836.895        True           0.400622           0.596033   
835  9277.905  1836.430        True           0.596033           1.761709   

     volume_increases  higher_lows  lower_highs  
831             Fa

### 3.

In [6]:
eth_prices["open_int_upward"] = np.where((eth_prices["open_int"] > 1.03 * eth_prices["open_int"].shift(1)), True, False)
eth_prices["open_int_downward"] = np.where((eth_prices["open_int"] < 0.97 * eth_prices["open_int"].shift(1)), True, False)
print(eth_prices.head())

                        time     open     high      low    close   Volume  \
7  2023-04-16 17:45:00-04:00  2147.65  2155.99  2144.91  2151.23  218.529   
8  2023-04-16 18:00:00-04:00  2152.26  2152.86  2147.06  2147.80  241.043   
9  2023-04-16 18:15:00-04:00  2147.45  2148.62  2143.10  2144.22   70.747   
10 2023-04-16 18:30:00-04:00  2144.44  2148.13  2142.02  2147.85   65.306   
11 2023-04-16 18:45:00-04:00  2148.71  2150.00  2146.43  2149.49   47.186   

     open_int       mid  price_flat  volume_inc_1st_pd  volume_inc_2nd_pd  \
7   11754.273  2150.450        True           1.383009           0.815844   
8   11724.362  2149.960        True           0.815844           0.743641   
9   11720.270  2145.860        True           0.743641           1.512772   
10  11729.325  2145.075        True           1.512772           1.103025   
11  11737.254  2148.215        True           1.103025           0.293504   

    volume_increases  higher_lows  lower_highs  open_int_upward  \
7      

In [7]:
eth_prices = eth_prices[eth_prices["volume_increases"] == True]
print(eth_prices.loc[:,["time", "volume_increases"]])

                         time  volume_increases
10  2023-04-16 18:30:00-04:00              True
20  2023-04-16 21:00:00-04:00              True
48  2023-04-17 04:00:00-04:00              True
56  2023-04-17 06:00:00-04:00              True
57  2023-04-17 06:15:00-04:00              True
..                        ...               ...
806 2023-04-25 01:30:00-04:00              True
807 2023-04-25 01:45:00-04:00              True
814 2023-04-25 03:30:00-04:00              True
819 2023-04-25 04:45:00-04:00              True
830 2023-04-25 07:30:00-04:00              True

[110 rows x 2 columns]


In [8]:
eth_prices["twap_buy_possible"] = np.where((eth_prices["volume_increases"] == True) \
    & (eth_prices["price_flat"] == True) \
    & (eth_prices["higher_lows"] == True) \
    & (eth_prices["open_int_upward"] == True), True, False)
eth_prices["twap_sell_possible"] = np.where((eth_prices["volume_increases"] == True) \
    & (eth_prices["price_flat"] == True) \
    & (eth_prices["lower_highs"] == True) \
    & (eth_prices["open_int_downward"] == True), True, False)
print(eth_prices.head(25))

                         time     open     high      low    close   Volume  \
10  2023-04-16 18:30:00-04:00  2144.44  2148.13  2142.02  2147.85   65.306   
20  2023-04-16 21:00:00-04:00  2101.95  2111.32  2093.11  2110.25  660.116   
48  2023-04-17 04:00:00-04:00  2112.21  2114.86  2107.59  2112.36  236.594   
56  2023-04-17 06:00:00-04:00  2110.67  2112.60  2106.10  2112.57  110.419   
57  2023-04-17 06:15:00-04:00  2112.50  2112.59  2104.57  2106.53  130.787   
58  2023-04-17 06:30:00-04:00  2106.38  2111.71  2104.56  2111.34  229.216   
59  2023-04-17 06:45:00-04:00  2111.02  2114.42  2109.98  2113.63   33.189   
60  2023-04-17 07:00:00-04:00  2113.94  2114.22  2111.35  2111.35   78.105   
71  2023-04-17 09:45:00-04:00  2090.11  2092.68  2084.37  2091.35  201.267   
78  2023-04-17 11:30:00-04:00  2091.38  2092.11  2083.22  2088.24  260.685   
96  2023-04-17 16:00:00-04:00  2097.93  2101.44  2097.04  2097.70   50.841   
107 2023-04-17 18:45:00-04:00  2099.78  2100.86  2097.54  2098.4

In [9]:
twaps_possible = eth_prices[(eth_prices["twap_buy_possible"] == True) | (eth_prices["twap_sell_possible"] == True)]
# twaps_possible = twaps_possible[["time", "open", "high", "low","twap_buy_possible",  "twap_sell_possible"]]
twaps_possible = twaps_possible[["time", "open","twap_buy_possible",  "twap_sell_possible"]]

print(twaps_possible.head(20))

Empty DataFrame
Columns: [time, open, twap_buy_possible, twap_sell_possible]
Index: []


##### -> No data emerging from the constraints, will refine to see if other combinations generate the scenarios